In [13]:
# Cell 1: Install and Import Dependencies
!pip install -q transformers datasets torch sentencepiece accelerate evaluate huggingface_hub
!pip install -q wandb  # for experiment tracking

import os
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
)
from datasets import load_dataset
import wandb
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


True

In [14]:
# Cell 2: Load and Prepare Dataset
# Load dataset from your Hugging Face repository
dataset = load_dataset("Mozes721/stock-crypto-weather-dataset", data_files="fine_tuning_data.json")

# Print dataset info
print("Dataset structure:", dataset)
print("\nSample example:", dataset['train'][0])

# Split dataset into train and validation
train_val = dataset['train'].train_test_split(test_size=0.1)
train_dataset = train_val['train']
val_dataset = train_val['test']

print(f"\nTraining examples: {len(train_dataset)}")
print(f"Validation examples: {len(val_dataset)}")

Generating train split: 145 examples [00:00, 1092.49 examples/s]

Dataset structure: DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 145
    })
})

Sample example: {'instruction': 'What is the current price of Bitcoin?', 'input': 'price=45000', 'output': 'The price of Bitcoin is 45000 USD'}

Training examples: 130
Validation examples: 15


In [15]:
# Cell 3: Initialize Model and Tokenizer
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Print model info
print(f"Model: {model_name}")
print(f"Model parameters: {model.num_parameters():,}")

Model: google/flan-t5-small
Model parameters: 76,961,152


In [16]:
# Cell 4: Data Preprocessing
def preprocess_function(examples):
    # Tokenize inputs
    inputs = examples['instruction']
    targets = examples['output']
    
    model_inputs = tokenizer(
        inputs,
        max_length=128,
        truncation=True,
        padding="max_length"
    )
    
    # Tokenize targets
    labels = tokenizer(
        targets,
        max_length=128,
        truncation=True,
        padding="max_length"
    )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
tokenized_train = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

tokenized_val = val_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=val_dataset.column_names
)

print("Sample tokenized input:", tokenizer.decode(tokenized_train[0]['input_ids']))
print("Sample tokenized target:", tokenizer.decode(tokenized_train[0]['labels']))

Map: 100%|██████████| 15/15 [00:00<00:00, 3999.91 examples/s]

Sample tokenized input: How much does Ethereum cost?</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
Sample tokenized target: Current Ethereum sits at 2890 USD</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [17]:
!pip install -q nbformat
# Training Arguments
training_args = Seq2SeqTrainingArguments(

    eval_strategy="epoch",  # Changed from evaluation_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=False,
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    push_to_hub=True,
    hub_model_id="Mozes721/crypto-stock-weather-agent",
    hub_token=os.getenv("HF_ACCESS_TOKEN"),
    report_to="wandb"
)

# Initialize wandb
wandb.init(
    project="crypto-stock-weather-agent",
    name="flan-t5-small",
    config={
        "model_name": "flan-t5-small-finetuned",
        "learning_rate": 2e-5,
        "batch_size": 4,
        "epochs": 3,
        "weight_decay": 0.01
    }
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


eval/loss,█▃▁
eval/runtime,█▁▄
eval/samples_per_second,▁█▅
eval/steps_per_second,▁█▅
train/epoch,▁▂▃▃▃▄▅▅▆▇▇██
train/global_step,▁▂▃▃▃▄▅▅▆▇▇██
train/grad_norm,▅▇█▅▆▄▄▁▂
train/learning_rate,█▇▆▅▅▄▃▂▁
train/loss,█▆▅▄▃▂▂▁▁
eval/loss,23.44195
eval/runtime,0.2766


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
# Create the trainer

from transformers import (
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Start training
trainer.train()

# Save the model
trainer.save_model("./results/flan-t5-small-finetuned")

# Push to Hub (this is automatic because we set push_to_hub=True in training_args)
# But we can also do it manually:
trainer.push_to_hub()


/var/folders/vk/476vpzr57lgbvz8vg4dbmtfc0000gn/T/ipykernel_69208/4171557761.py:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/Users/richardtaujenis/rtaujenis/personal_projects/LlamaRAG/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,32.846100,33.556351
2,28.503600,26.897081
3,25.789700,24.864452


/Users/richardtaujenis/rtaujenis/personal_projects/LlamaRAG/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/richardtaujenis/rtaujenis/personal_projects/LlamaRAG/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Mozes721/crypto-stock-weather-agent/commit/9cf88cbc400eac7ff11535f684c82b4093cf603b', commit_message='End of training', commit_description='', oid='9cf88cbc400eac7ff11535f684c82b4093cf603b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Mozes721/crypto-stock-weather-agent', endpoint='https://huggingface.co', repo_type='model', repo_id='Mozes721/crypto-stock-weather-agent'), pr_revision=None, pr_num=None)

In [19]:
# Test the fine-tuned model
from transformers import pipeline

# Load the fine-tuned model from Hugging Face Hub
model_id = "Mozes721/crypto-stock-weather-agent"
generator = pipeline(
    "text2text-generation",
    model=model_id,
    max_length=256,  # Longer to accommodate full templates
    num_return_sequences=1,
    temperature=0.1,  # More consistent outputs
    do_sample=False  # More deterministic
)

# Test examples with values included in the instruction
test_queries = [
    {
        "instruction": "What is the current price of Bitcoin?",
        "input": "price=45000"
    },
    {
        "instruction": "How's the weather in Riga?",
        "input": "temp=15 weather=partly cloudy wind_speed=4.8"
    },
    {
        "instruction": "Could you tell me the weather in Warsaw",
        "input": "temp=20 weather=partly cloudy wind_speed=2.1",
    },
    {
        "instruction": "What's the current price of Tesla stock?",
        "input": "price=250.75"
    },
    {
        "instruction": "Tell me the price of Ethereum",
        "input": "price=2800"
    },
    {
        "instruction": "What's the weather like in Tokyo?",
        "input": "temp=22 weather=clear sky wind_speed=5.2"
    }
]

print("Testing the model with different queries:\n")
for query in test_queries:
    # Combine instruction and input for the model
    full_prompt = f"{query['instruction']} | Values: {query['input']}"
    response = generator(full_prompt)
    generated_text = response[0]['generated_text']
    print(f"Query: {full_prompt}")
    print(f"Model Response: {generated_text}")
    print("---\n")


Device set to use mps:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Testing the model with different queries:



The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Query: What is the current price of Bitcoin? | Values: price=45000
Model Response: Bitcoin is a cryptocurrency that has a price of $45000
---



The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

Query: Could you tell me the weather in Warsaw | Values: temp=20 weather=partly cloudy wind_speed=2.1
Model Response: Weather in Warsaw is moderately cloudy.
---



The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Query: What's the current price of Tesla stock? | Values: price=250.75
Model Response: The price of a Tesla stock is 250.75
---



The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Query: Tell me the price of Ethereum | Values: price=2800
Model Response: Ethereum is a cryptocurrency with a price of 2800.
---

Query: What's the weather like in Tokyo? | Values: temp=22 weather=clear sky wind_speed=5.2
Model Response: Tokyo has a temperature of 22 degrees Fahrenheit and a wind speed of 5.2 degrees Fahrenheit.
---



In [1]:
# The instruction part of your query
instruction = "Could you tell me the weather in Warsaw"

# The values part
input_values = "temp=20 weather=partly cloudy wind_speed=2.1"

# Combine them EXACTLY as the model was trained
# (A simple space is the most common way)
formatted_query = f"{instruction} {input_values}"

# Now, pass this correctly formatted string to your pipeline
response = generator(formatted_query)

print(f"Query: {formatted_query}")
print(f"Model Response: {response[0]['generated_text']}")

NameError: name 'generator' is not defined